# How to train your own word vector embeddings with Keras

Many tasks require embeddings or domain-specific vocabulary that pre-trained models based on a generic corpus may not represent well or at all. Standard word2vec models are not able to assign vectors to out-of-vocabulary words and instead use a default vector that reduces their predictive value.

E.g., when working with industry-specific documents, the vocabulary or its usage may change over time as new technologies or products emerge. As a result, the embeddings need to evolve as well. In addition, corporate earnings releases use nuanced language not fully reflected in Glove vectors pre-trained on Wikipedia articles.

We will illustrate the word2vec architecture using the keras library that we will introduce in more detail in the next chapter and the more performant gensim adaptation of the code provided by the word2vec authors. 

To illustrate the word2vec network architecture, we use the Financial News data that we first introduced in chapter 14 on Topic Modeling. 

## Imports

If you have not yet install `TensorFlow 2`, uncomment and run one of the following, the first if you have a GPU and the second otherwise.

In [ ]:
# !conda install -n ml4t-text tensorflow-gpu -y

In [ ]:
# !conda install -n ml4t-text tensorflow -y

In [1]:
%matplotlib inline
from time import time
from collections import Counter
from pathlib import Path
import pandas as pd
import numpy as np
from numpy.linalg import norm
from scipy.spatial.distance import cdist, cosine

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Dot, Embedding
from tensorflow.keras.preprocessing.sequence import skipgrams, make_sampling_table
from tensorflow.keras.callbacks import Callback, TensorBoard

### Settings

In [2]:
sns.set_style('whitegrid')
pd.set_option('float_format', '{:,.2f}'.format)
np.random.seed(42)

### Paths

In [3]:
news_path = Path('data', 'fin_news')
data_path = news_path / 'data'
analogy_path = Path('data', 'analogies', 'analogies-en.txt')

In [4]:
def format_time(t):
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h:02.0f}:{m:02.0f}:{s:02.0f}'

## `word2vec` - skipgram Architecture using Keras

### Settings

In [5]:
LANGUAGE = 'en'
SAMPLE_SIZE=.5              # portion of sentences to use for model
NGRAMS = 3                  # Longest ngram in text
MIN_FREQ = 10

In [6]:
SAMPLING_FACTOR = 1e-4
WINDOW_SIZE = 3
EMBEDDING_SIZE = 300
EPOCHS = 1
BATCH_SIZE = 2500

In [7]:
# Set up validation
VALID_SET = 10      # Random set of words to get nearest neighbors for
VALID_WINDOW = 150  # Most frequent words to draw validation set from
NN = 10             # Number of nearest neighbors for evaluation

valid_examples = np.random.choice(VALID_WINDOW, size=VALID_SET, replace=False)

In [8]:
FILE_NAME = f'articles_{NGRAMS}_grams.txt'
file_path = data_path / FILE_NAME

In [9]:
tb_path = news_path / 'tensorboard'
if not tb_path.exists():
    tb_path.mkdir(parents=True, exist_ok=True)

### Build Data Set

#### Tokens to ID

1. Extract the top *n* most common words to learn embeddings
2. Index these *n* words with unique integers
3. Create an `{index: word}` dictionary
4. Replace the *n* words with their index, and a dummy value `UNK` elsewhere

In [10]:
sentences = file_path.read_text().split('\n')
words = ' '.join(np.random.choice(sentences, size=int(.5*len(sentences)), replace=False)).split()

In [11]:
# Get (token, count) tuples for tokens meeting MIN_FREQ 
token_counts = [t for t in Counter(words).most_common() if t[1] >= MIN_FREQ]
tokens, counts = list(zip(*token_counts))

# create id-token dicts & reverse dicts
id_to_token = pd.Series(tokens, index=range(1, len(tokens) + 1)).to_dict()
id_to_token.update({0: 'UNK'})
token_to_id = {t:i for i, t in id_to_token.items()}
data = [token_to_id.get(word, 0) for word in words]

In [12]:
vocab_size = len(token_to_id)

In [13]:
vocab_size

59617

In [14]:
s = pd.Series(data).value_counts().reset_index()
s.columns = ['id', 'count']
s['token'] = s.id.map(id_to_token)

In [15]:
s.sort_values('count', ascending=False).head(10)

,id,count,token
0,0,411047,UNK
1,1,171157,company
2,2,168115,million
3,3,144088,said
4,4,110774,year
5,5,106468,quarter
6,6,82508,financial
7,7,78595,reuters
8,8,73047,net
9,9,72307,new


In [16]:
s['count'].sum()

17391040

In [17]:
s.sort_values('id').token.dropna().to_csv(tb_path / 'meta.tsv', index=False)

#### Analogies to ID

In [18]:
def get_analogies():
    df = pd.read_csv(analogy_path, header=None, squeeze=True)
    categories = df[df.str.startswith(':')]
    analogies = df[~df.str.startswith(':')].str.split(expand=True)
    analogies.columns = list('abcd')
    return analogies

In [19]:
analogies = get_analogies()
analogies.head()

,a,b,c,d
1,athens,greece,baghdad,iraq
2,athens,greece,bangkok,thailand
3,athens,greece,beijing,china
4,athens,greece,berlin,germany
5,athens,greece,bern,switzerland


In [20]:
analogies_id = analogies.apply(lambda x: x.map(token_to_id))
analogies_id.notnull().all(1).sum()/len(analogies_id)

0.7269218651543793

### Generate Sampling Probabilities

There is an alternative, faster scheme than the traditional SoftMax loss function called [Noise Contrastive Estimation (NCE)](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

Instead of getting the softmax probability for all possible context words, randomly sample 2-20 possible context words and evaluate the probability only for these.

**SAMPLING_FACTOR**: used for generating the `sampling_table` argument for `skipgrams`. 

`sampling_table[i]` is the probability of sampling the word i-th most common word in a dataset

The sampling probabilities are generated according
to the sampling distribution used in word2vec:

$p(\text{word}) = \min\left(1, \frac{\sqrt{\frac{\text{word frequency}}{\text{sampling factor}}}}{\frac{\text{word frequency}}{\text{sampling factor}}}\right)$

In [ ]:
df = s['count'].to_frame('freq')
factors = [1, 1e-2, 1e-4, 1e-6, 1e-8]
for f in factors:
    sf = make_sampling_table(vocab_size, sampling_factor=f)
    df[f] = df.freq.mul(sf)
df.loc[:, factors].plot(logy=True, xlim=(0, 60000));

In [ ]:
sampling_table = make_sampling_table(vocab_size, sampling_factor=SAMPLING_FACTOR/10)

In [ ]:
pd.Series(sampling_table).plot(title='Skip-Gram Sampling Probabilities')
plt.tight_layout();

### Generate target-context word pairs

In [ ]:
pairs, labels = skipgrams(sequence=data,
                          vocabulary_size=vocab_size,
                          window_size=WINDOW_SIZE,
                          sampling_table=sampling_table,
                          negative_samples=1.0,
                          shuffle=True)

print('{:,d} pairs created'.format(len(pairs)))

In [ ]:
target_word, context_word = np.array(pairs, dtype=np.int32).T
labels = np.array(labels, dtype=np.int8)
del pairs

In [ ]:
target_word[:5]

In [ ]:
df = pd.DataFrame({'target': target_word[:5], 
                   'context': context_word[:5], 
                   'label': labels[:5]})
df

In [ ]:
pd.Series(labels).value_counts()

In [ ]:
with pd.HDFStore(news_path / 'data.h5') as store:
    store.put('id_to_token', pd.Series(id_to_token))
    store.put('pairs', pd.DataFrame({'target' : target_word,
                                     'context': context_word, 
                                     'labels': labels}))

In [21]:
with pd.HDFStore(news_path / 'data.h5') as store:
    id_to_token = store['id_to_token']
    pairs = store['pairs']
target_word, context_word, labels = pairs.target, pairs.context, pairs.labels

### Define Keras Model Components

#### Scalar Input Variables

In [22]:
input_target = Input((1,), name='target_input')
input_context = Input((1,), name='context_input')

#### Shared Embedding Layer

In [23]:
embedding = Embedding(input_dim=vocab_size,
                      output_dim=EMBEDDING_SIZE,
                      input_length=1,
                      name='embedding_layer')

In [24]:
target = embedding(input_target)
target = Reshape((EMBEDDING_SIZE, 1), name='target_embedding')(target)

context = embedding(input_context)
context = Reshape((EMBEDDING_SIZE, 1), name='context_embedding')(context)

#### Create Similarity Measure

In [25]:
dot_product = Dot(axes=1)([target, context])
dot_product = Reshape((1,), name='similarity')(dot_product)

#### Sigmoid Output Layer

In [26]:
output = Dense(units=1, activation='sigmoid', name='output')(dot_product)

#### Compile Training Model

In [27]:
model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

#### Display Architecture

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
target_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
context_input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 1, 300)       17885100    target_input[0][0]               
                                                                 context_input[0][0]              
__________________________________________________________________________________________________
target_embedding (Reshape)      (None, 300, 1)       0           embedding_layer[0][0]        

#### Validation Model

In [29]:
similarity = Dot(normalize=True, 
                 axes=1, 
                 name='cosine_similarity')([target, context])

In [30]:
# create a secondary validation model to run our similarity checks during training
validation_model = Model(inputs=[input_target, input_context], outputs=similarity)

In [31]:
validation_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
target_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
context_input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 1, 300)       17885100    target_input[0][0]               
                                                                 context_input[0][0]              
__________________________________________________________________________________________________
target_embedding (Reshape)      (None, 300, 1)       0           embedding_layer[0][0]      

![Keras Graph](https://s3.amazonaws.com/applied-ai/images/keras_graph_tensorboard.png)

### Create Keras Callbacks

####  Nearest Neighors & Analogies

In [32]:
test_set = analogies_id.dropna().astype(int)
a, b, c, actual = test_set.values.T
actual = actual.reshape(-1, 1)
n_analogies = len(actual)

In [33]:
class EvalCallback(Callback):
        
    def on_train_begin(self, logs={}):
        self.eval_nn()
        self.test_analogies()

    def on_train_end(self, logs={}):
        self.eval_nn()

    def on_epoch_end(self, batch, logs={}):
        self.test_analogies()

    @staticmethod
    def test_analogies():
        print('\nAnalogy Accuracy:\n\t', end='')
        embeddings = embedding.get_weights()[0]
        target = embeddings[c] + embeddings[b] - embeddings[a]
        neighbors = np.argsort(cdist(target, embeddings, metric='cosine'))
        match_id = np.argwhere(neighbors == actual)[:, 1]
        print('\n\t'.join(['Top {}: {:.2%}'.format(i, (match_id < i).sum() / n_analogies) for i in [1, 5, 10]]))

    def eval_nn(self):
        print('\n{} Nearest Neighbors:'.format(NN))
        for i in range(VALID_SET):
            valid_id = valid_examples[i]
            valid_word = id_to_token[valid_id]
            similarity = self._get_similiarity(valid_id).reshape(-1)
            nearest = (-similarity).argsort()[1:NN + 1]
            neighbors = [id_to_token[nearest[n]] for n in range(NN)]
            print('{}:\t{}'.format(valid_word, ', '.join(neighbors)))            
        
    @staticmethod
    def _get_similiarity(valid_word_idx):
        target = np.full(shape=vocab_size, fill_value=valid_word_idx)
        context = np.arange(vocab_size)
        return validation_model.predict([target, context])


evaluation = EvalCallback()

#### Tensorboard Callback

Currently tensorflow has a [bug](https://github.com/tensorflow/tensorflow/issues/32902) that prevents metadata from working. The GitHub issue points to a simple fix that you can apply to the tensorflow source code, just search for the culprit line and change accordingly until a later release remedies this problem. You will have to install with `pip` for this purpose.

In [34]:
tensorboard = TensorBoard(log_dir=str(tb_path),
                          write_graph=True,
                          embeddings_freq=1,
                          embeddings_metadata={'embedding_layer': 
                                               str(tb_path / 'meta.tsv')})

### Train Model

In [35]:
loss = model.fit(x=[target_word, context_word],
                 y=labels,
                 shuffle=True,
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[evaluation, tensorboard] # uncomment if tensorboard bug is fixed
#                  callbacks=[evaluation]
                )


10 Nearest Neighbors:
loss:	gernot_heller, san_ramon, stamos, three_fold, facilitate, cheating, well_timed, relocations, imperialism, philipp
non:	robin, aoun, michael_froman, leaves_downing_street, falzone, sirius_xm_holdings, vivek_mishra, taxman, estimation_mineral_reserves, cornerstone
service:	tahoe, fraport, aligns, saleable_equivalent, celldex_therapeutics, belgravia, actuarial_gains_losses, deficit, predictability, endo_bariatric
announced:	vitalhub, plutonic_gold, controllers, reproductive_medicine, life_threatening_diseases, mellanox, mexus, purging, doug_mcmillon, lynne
ebitda:	clarifying, leftist_front_runner_andres, trimming, committee_medicinal, reflecting, sarajevo, momentous, bot, assurant, alien
december:	fiancee_meghan_markle, spare_parts, bgc, gina_cherelus, pki, peruvian, luciano_costa, fabre, flynn, nioc
securities:	pandora_box, scholl, crippling, human_trafficking, fy_fy, jostle, slope, replacements, babcock_wilcox_enterprises, writing_maria_kiselyova
result:	veg

In [ ]:
model.save(str(news_path / 'keras' / 'skipgram_model.h5'))

### Visualize Embeddings using Tensorboard

Load the embeddings metadata using the `load` option to view embedding labels and see [tutorial](https://www.tensorflow.org/tensorboard/get_started) for usage instructions.

In [36]:
%load_ext tensorboard

In [37]:
%tensorboard --logdir data/fin_news/tensorboard/train

## Resources

- [Distributed representations of words and phrases and their compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
- [Efficient estimation of word representations in vector space](https://arxiv.org/pdf/1301.3781.pdf?)
- [Sebastian Ruder's Blog](http://ruder.io/word-embeddings-1/)